# B3 - tratamento e segregação por tipo de títulos

## Código para gerar o json B3 a partir de planilha Excel

In [3]:
import pandas as pd
import json

# --- 1. Carregar o Excel ---
arquivo = "B3_Empresas_Setor_20260206.xlsx"

# Lê apenas o intervalo desejado (B3:H372)
df = pd.read_excel(
    arquivo,
    sheet_name="Setor",  # nome da aba
    usecols="B:H",
    skiprows=2,       # pula até a linha 3
    nrows=370         # garante leitura até a linha 372
)

# Renomear colunas para facilitar
df.columns = [
    "Ticker_BEEST",  # Coluna B
    "Coluna_C",      # Coluna C (não usada)
    "Coluna_D",      # Coluna D (não usada)
    "Setor",         # Coluna E
    "Empresa",       # Coluna F
    "Ticker_B3",     # Coluna G
    "Segmento_B3"    # Coluna H
]

# Manter apenas as colunas relevantes
df = df[["Ticker_BEEST", "Setor", "Empresa", "Ticker_B3", "Segmento_B3"]]

def _clean(value: object) -> str:
    """
    Normaliza valores do Excel para texto limpo.

    Converte valores nulos (NaN) em string vazia e remove espaços
    em branco nas extremidades de valores válidos.

    :param value: Valor original vindo do DataFrame.
    :type value: object
    :return: Valor convertido para string limpa ou "" se nulo.
    :rtype: str
    """
    if pd.isna(value):
        return ""
    return str(value).strip()

# --- 2. Construir o dicionário no formato solicitado ---
# Estrutura: {BEEST: {Setor: {Ticker_B3: {Empresa, Segmento_B3}}}}
ticker_beest = {}

for _, row in df.iterrows():
    beest = _clean(row["Ticker_BEEST"])
    setor = _clean(row["Setor"])
    ticker_b3 = _clean(row["Ticker_B3"])
    empresa = _clean(row["Empresa"])
    segmento_b3 = _clean(row["Segmento_B3"])

    # Ignorar linhas vazias ou incompletas
    if not beest or not setor or not ticker_b3:
        continue

    # Construir o dicionário aninhado
    ticker_beest.setdefault(beest, {}).setdefault(setor, {})[ticker_b3] = {
        "Empresa": empresa,
        "Segmento_B3": segmento_b3
    }

# --- 3. Salvar em JSON ---
with open("Ticker_beest.json", "w", encoding="utf-8") as f:
    json.dump(ticker_beest, f, ensure_ascii=False, indent=4)

print("Arquivo JSON gerado com sucesso!")

Arquivo JSON gerado com sucesso!


## Código para gerar o json Yahoo Finance a partir do json B3

In [ ]:
import json
from pathlib import Path

def _resolve_path(filename: str) -> Path: # _antes indica função privada
    """
    Verificar o caminho de um arquivo, verificando locais padrão.

    Tenta o caminho fornecido diretamente e, se não existir, procura na pasta
    ``utils``. Lança ``FileNotFoundError`` se não encontrar.

    :param filename: Nome do arquivo a localizar.
    :type filename: str
    :return: Caminho resolvido para o arquivo.
    :rtype: Path
    """
    candidate = Path(filename)  # Caminho fornecido diretamente
    if candidate.exists():
        return candidate  # Verifica o caminho fornecido diretamente
    fallback = Path("utils") / filename
    if fallback.exists():
        return fallback  # Verifica o caminho dentro da pasta "utils"
    raise FileNotFoundError(f"Arquivo não encontrado: {filename}")

def _resolve_any(filenames: list[str]) -> Path:
    """
    Verifica o primeiro arquivo existente dentre uma lista de nomes.
    Tenta cada nome na ordem fornecida e retorna o caminho do primeiro que existir.
    Lança ``FileNotFoundError`` se nenhum dos arquivos for encontrado.
    """
    last_error = None # Guarda o último erro para relançar se nenhum arquivo for encontrado
    for name in filenames:
        try:
            return _resolve_path(name) # Tenta resolver cada nome na ordem
        except FileNotFoundError as exc:
            last_error = exc
    raise last_error if last_error else FileNotFoundError("Arquivo não encontrado")

# --- 1. Carregar arquivos ---

# Tenta ambos os nomes comuns para o arquivo B3, garantindo compatibilidade com versões anteriores
path_b3 = _resolve_any(["tickers_b3_4_11.json", "tickers_b3_3_4_11.json"])
# O arquivo BEEST é esperado com nome fixo, mas também pode ser verificado em "utils"
path_beest = _resolve_path("Ticker_beest.json")

# Carrega os dados dos arquivos JSON que tem a estrutura {Ticker_B3: Nome_Empresa}
with path_b3.open("r", encoding="utf-8") as f:
    b3_map = json.load(f)  # {"PETR4": "PETROLEO BRASILEIRO...", ...}

# Carrega os dados do BEEST, que tem a estrutura {BEEST: {Setor: {Ticker_B3: {Empresa, Segmento_B3}}}}
with path_beest.open("r", encoding="utf-8") as f:
    ticker_beest = json.load(f)

# --- 2. Criar índice por prefixo (4 letras) ---
prefix_index = {} # {"PETR": ["PETR3", "PETR4", "PETR11"], "VALE": ["VALE3"], ...}
for ticker_b3 in b3_map.keys(): # "PETR4", "VALE3", etc.
    prefix = ticker_b3[:4] # "PETR", "VALE", etc.
    # Agrupa os tickers por prefixo para facilitar a correspondência
    prefix_index.setdefault(prefix, []).append(ticker_b3) # Ex: "PETR" -> ["PETR3", "PETR4", "PETR11"]

# --- 3. Atualizar tickers para nomenclatura Yahoo Finance ---
# Agora preserva TODOS os sufixos encontrados (ex.: PETR3, PETR4, PETR11)
ticker_beest_yf = {} # Estrutura final: {BEEST: {Setor: {Ticker_YF: {Empresa, Segmento_B3}}}}
missing = set() # Para rastrear tickers base que não encontraram correspondência no B3

# Itera sobre a estrutura original do BEEST e tenta mapear cada ticker 
# base para os tickers do B3 usando o índice de prefixo
for grupo, setores in ticker_beest.items(): # "BEEST1", "BEEST2", etc.
    ticker_beest_yf.setdefault(grupo, {}) # Inicializa o grupo no resultado final
    # Itera sobre os setores dentro de cada grupo
    for setor, tickers in setores.items():
        ticker_beest_yf[grupo].setdefault(setor, {}) # Inicializa o setor no resultado final
        # Itera sobre os tickers base (chaves do BEEST) dentro de cada setor
        for base_ticker, info in tickers.items(): # "PETR", "VALE", etc.
            base_ticker = str(base_ticker).strip() # Garante que o ticker base seja uma string limpa
            candidatos = sorted(prefix_index.get(base_ticker, [])) # Obtém os tickers do B3 que correspondem ao prefixo do ticker base
            # Se não houver candidatos, registra o ticker base como faltante e mantém a estrutura original
            if not candidatos:
                missing.add(base_ticker) # Registra o ticker base que não encontrou correspondência
                novo_info = dict(info) # Cria uma cópia do dicionário de informações para evitar mutações
                # Mantém o ticker original do BEEST, mas adiciona um sufixo para indicar que é um caso sem correspondência
                ticker_beest_yf[grupo][setor][base_ticker] = novo_info
                continue
            # Para cada ticker candidato encontrado, cria uma nova entrada no resultado final com o sufixo ".SA" e atualiza o nome da empresa usando o mapeamento do B3
            for escolhido in candidatos:
                novo_ticker = f"{escolhido}.SA" # Ex: "PETR4" -> "PETR4.SA"
                novo_info = dict(info) # Cria uma cópia do dicionário de informações para evitar mutações
                # Atualiza o nome da empresa com base no arquivo B3
                if escolhido in b3_map:
                    novo_info["Empresa"] = b3_map[escolhido] # Ex: "PETR4" -> "PETROLEO BRASILEIRO S.A. - PETROBRAS"
                # Adiciona a nova entrada ao resultado final, preservando o setor e as informações originais, mas com o ticker atualizado para o formato do Yahoo Finance
                ticker_beest_yf[grupo][setor][novo_ticker] = novo_info

# --- 4. Salvar em JSON ---
saida = Path("ticker_besst_yf.json")
with saida.open("w", encoding="utf-8") as f:
    json.dump(ticker_beest_yf, f, ensure_ascii=False, indent=4)

print("Arquivo gerado:", saida)
if missing:
    print("Tickers base sem correspondência:", ", ".join(sorted(missing)))

Arquivo gerado: ticker_besst_yf.json
Tickers base sem correspondência: AGRU, AURA, BSCS, CTBA, EQMA, G2DI, GOLL, INBR, IVPR, JBSS, LIGH, LTEL, LTLA, MCRJ, MRSA, PLSC, PMSP, PRMN, RBRA
